# Test Shallow Water Physics on a mesh
Forward Euler and forward backward integrators are tested and compared on a hexagonal mesh loaded from a file

In [1]:
include("mode_forward.jl")
include("mode_init.jl")
include("visualization.jl")

using PyPlot
using PyCall

patch = pyimport("matplotlib.patches")
col = pyimport("matplotlib.collections")
animation = pyimport("matplotlib.animation")
ipydisplay = pyimport("IPython.display")

PyObject <module 'IPython.display' from '/home/rrs/anaconda3/envs/MPAS_Ocean/lib/python3.8/site-packages/IPython/display.py'>

## load mesh from a file

In [3]:
myMPAS_O = MPAS_Ocean(false, 
                "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                "base_mesh.nc",
                "mesh.nc")
myMPAS_O.myNamelist

Namelist(1000.0, "uniform", #undef, false, false, true, "Periodic", "Forward_Backward", #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, 0.0, 0.0001, 2.0000000000000002e-11, 10.0, 1000.0, 0.001, "default", "centered", false, 100.0, 1.0e6, 2.2360679774997896e6, 180.0, 0.0, 500.0, 500.0, 1.0, 0.0, 0.5, 12500.0)

## define dt, duration of the simulation, number of snapshots of the sea surface height to record, and arrays for recording it

In [4]:
# calculate dt based on CFL condition
dt = 0.1 * minimum(myMPAS_O.dcEdge) / sqrt( gravity  * maximum(myMPAS_O.bottomDepth) )

nFrames = 200

simTimeBetweenFrames = 3*dt

"dt: ", dt

("dt: ", 1.0101525445522077)

# Run the simulation

## gaussian initial condition function

In [5]:
function gaussianInit!(myMPAS_O)
    # define gaussian initial condition
    for iCell in 1:myMPAS_O.nCells
        myMPAS_O.sshCurrent[iCell] = exp(
            (
                -(myMPAS_O.yCell[iCell] - 20000)^2
                -(myMPAS_O.xCell[iCell] - 30000)^2
            ) / ( 1.0e7 )
        )
    end

    myMPAS_O.normalVelocityCurrent = zeros(myMPAS_O.nEdges)
end

gaussianInit! (generic function with 1 method)

## integrate with forward euler

In [6]:
# solve it, integrate!
## forward euler
gaussianInit!(myMPAS_O)

sshOverTimeFE = zeros(Float64, (nFrames, myMPAS_O.nCells))

for frame in 1:nFrames
    forwardEuler!(myMPAS_O, dt, simTimeBetweenFrames)
    sshOverTimeFE[frame,:] = myMPAS_O.sshCurrent[:]
end


## animate forward euler solution

In [7]:
fig = figure()

cMax = maximum(abs.(sshOverTimeFE))

_, ax, _, collection = heatMapMesh(myMPAS_O, sshOverTimeFE[1,:], fig=fig, cMin=-cMax, cMax=cMax)

function nextFrame(j)
    i = j + 1
    
    collection.set_array(sshOverTimeFE[i,:])
    
    return ax
end

anim = animation.FuncAnimation(fig, nextFrame, frames=nFrames)

PyObject <matplotlib.animation.FuncAnimation object at 0x7fa7506c5f10>

In [8]:
ipydisplay.HTML(anim.to_html5_video())

PyObject <IPython.core.display.HTML object>

In [9]:
# anim.save("plots/test1/forwardeuler_ssh_test1.mp4")
# ipydisplay.Video("plots/test1/forwardeuler_ssh_test1.mp4")

## reset simulation and run with forward backward solver

In [10]:
# reset
gaussianInit!(myMPAS_O)

sshOverTimeFB = zeros(Float64, (nFrames, myMPAS_O.nCells))

for frame in 1:nFrames
    forwardBackward!(myMPAS_O, dt, simTimeBetweenFrames)
    sshOverTimeFB[frame,:] = myMPAS_O.sshCurrent[:]
end

# Compare forward euler and forward backward with side-by-side animations

In [11]:
fig = figure()

ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

cMax1 = maximum(abs.(sshOverTimeFE))
cMax2 = maximum(abs.(sshOverTimeFB))
cMax3 = maximum(abs.(sshOverTimeFE .- sshOverTimeFB))


_, _, _, col1 = heatMapMesh(myMPAS_O, sshOverTimeFE[1,:]; fig=fig, ax=ax1, cMin=-cMax1, cMax=cMax1)
ax1.set_title("Forward Euler")
ax1.axis("off")

_, _, _, col2 = heatMapMesh(myMPAS_O, sshOverTimeFB[1,:]; fig=fig, ax=ax2, cMin=-cMax2, cMax=cMax2)
ax2.set_title("Forward Backward")
ax2.axis("off")

_, _, _, col3 = heatMapMesh(myMPAS_O, sshOverTimeFE[1,:] .- sshOverTimeFB[1,:]; fig=fig, ax=ax3, cMin=-cMax3, cMax=cMax3)
ax3.set_title("Difference")
ax3.axis("off")


tight_layout()

function nextFrame(j)

    i = j+1

    col1.set_array(sshOverTimeFE[i,:])
    col2.set_array(sshOverTimeFB[i,:])
    col3.set_array(sshOverTimeFE[i,:] - sshOverTimeFB[i,:])

    fig.suptitle("frame $j of $nFrames")

    return [ax1, ax2, ax3]

end

anim = animation.FuncAnimation(fig, nextFrame, frames=nFrames)

PyObject <matplotlib.animation.FuncAnimation object at 0x7fa728eb6100>

In [12]:
ipydisplay.HTML(anim.to_html5_video())

PyObject <IPython.core.display.HTML object>

In [18]:
# anim.save("plots/test2/forwardeuler_vs_forwardbackward.mp4")
# ipydisplay.Video("plots/test2/forwardeuler_vs_forwardbackward.mp4")